# Цели
Здесь я собираюсь сделать модель для предсказания цены закрытия акции по информации за $N$ предыдущих дней. 
А так же проверить следущие гипотезы:
1. Рынок приспосабливается к стратегиям. Иными словами, чем дальше период на котором обучалась модель от цели предсказания, тем ниже точность предсказания.
2. Внешние данны важны. Иными словами, предсказания котировок с ипользованием внешних текстов получаются точнее, чем без них.

В роли внешних текстов используются [новости с MOEX](https://www.moex.com/ru/news/?ncat=106). А точнее, их заголовки.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer

# Обработка данных   
## Исходные данные
Здесь мы собираем данные как есть

### Акции

In [2]:
path_to_tools = 'data/tools/list_tools.txt'
df_tools = pd.read_csv(path_to_tools,sep='\t')
df_tools.head()

,SECID,BOARDID,SHORTNAME,PREVPRICE,LOTSIZE,FACEVALUE,STATUS,BOARDNAME,DECIMALS,SECNAME,...,PREVDATE,ISSUESIZE,ISIN,LATNAME,REGNUMBER,PREVLEGALCLOSEPRICE,CURRENCYID,SECTYPE,LISTLEVEL,SETTLEDATE
0,ABIO,TQBR,iАРТГЕН ао,73.50,10,0.10,A,Т+: Акции и ДР - безадрес.,2,"ПАО ""Артген""",...,2024-12-12,92645451,RU000A0JNAB6,ARTGEN ao,1-01-08902-A,74.420,SUR,1,2,2024-12-16
1,ABRD,TQBR,АбрауДюрсо,165.00,10,1.00,A,Т+: Акции и ДР - безадрес.,1,Абрау-Дюрсо ПАО ао,...,2024-12-12,98000184,RU000A0JS5T7,Abrau-Durso ao,1-02-12500-A,165.000,SUR,1,3,2024-12-16
2,ACKO,TQBR,АСКО ао,3.58,100,1.00,A,Т+: Акции и ДР - безадрес.,2,АСКО ПАО ао,...,2024-12-12,536000000,RU000A0JXS91,ASKO ao,1-01-52065-Z,3.580,SUR,1,3,2024-12-16
3,AFKS,TQBR,Система ао,11.93,100,0.09,A,Т+: Акции и ДР - безадрес.,3,"АФК ""Система"" ПАО ао",...,2024-12-12,9650000000,RU000A0DQZE3,AFK Sistema,1-05-01669-A,11.932,SUR,1,1,2024-12-16
4,AFLT,TQBR,Аэрофлот,48.54,10,1.00,A,Т+: Акции и ДР - безадрес.,2,Аэрофлот-росс.авиалин(ПАО)ао,...,2024-12-12,3975771215,RU0009062285,Aeroflot,1-01-00010-A,48.550,SUR,1,1,2024-12-16



### Свечи

In [3]:
path_to_all_tools_candles = 'data/candles'
dfs = []
for path_to_candles in Path(path_to_all_tools_candles).glob('*'):
    df_candles = pd.read_csv(path_to_candles,sep='\t')
    df_candles['SECID'] = path_to_candles.name[:-6]
    dfs.append(df_candles)
df_candles = pd.concat(dfs)
df_candles.head()

C:\Users\user\AppData\Local\Temp\ipykernel_19244\2080516023.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_candles = pd.concat(dfs)


,begin,end,open,high,low,close,value,volume,SECID
0,2014-06-09 00:00:00,2014-06-09 23:59:59,15.508,15.508,14.801,15.000,22579.4,1500.0,ABIO
1,2014-06-10 00:00:00,2014-06-10 23:59:59,14.871,15.153,14.871,14.881,25469.9,1700.0,ABIO
2,2014-06-11 00:00:00,2014-06-11 23:59:59,15.000,15.000,14.999,14.999,2999.9,200.0,ABIO
3,2014-06-16 00:00:00,2014-06-16 23:59:59,15.468,15.468,15.000,15.000,24522.3,1600.0,ABIO
4,2014-06-17 00:00:00,2014-06-17 23:59:59,15.100,15.120,15.100,15.120,738392.0,48900.0,ABIO



### Текст

In [4]:
path_to_text = 'data/texts/moex_title.txt'
df_text = pd.read_csv(path_to_text)
df_text.head()

,id,tag,title,published_at,modified_at
0,75803,site,"О внесении изменений в Список ценных бумаг, до...",2024-12-13 17:34:03,2024-12-13 14:34:05
1,75802,site,"Цена исполнения фьючерсов на сахар, декабрь 2024",2024-12-13 17:30:45,2024-12-13 17:30:45
2,75801,site,Дополнительные условия проведения торгов отдел...,2024-12-13 17:25:56,2024-12-13 17:25:56
3,75799,site,"13.12.2024, 17-24 (мск) изменены значения нижн...",2024-12-13 17:24:00,2024-12-13 17:25:10
4,75797,site,"13.12.2024, 17-16 (мск) изменены значения верх...",2024-12-13 17:16:09,2024-12-13 17:20:04


## Входные данные
Здесь мы приводим данные к универсальному виду, который и будем в дальнейшем передавать в модель для дальнейшей предобработки

### Акционные данные

In [5]:
df_tools = df_tools[['SECID','BOARDID','SHORTNAME','SECNAME','LOTSIZE','SECTYPE','LISTLEVEL']]
df_tools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SECID      260 non-null    object
 1   BOARDID    260 non-null    object
 2   SHORTNAME  260 non-null    object
 3   SECNAME    260 non-null    object
 4   LOTSIZE    260 non-null    int64 
 5   SECTYPE    260 non-null    object
 6   LISTLEVEL  260 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 14.3+ KB


In [6]:
df_candles['day_date'] = df_candles['begin'].map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d')) 
df_candles = df_candles[['SECID','day_date','close','value','volume']]
df_candles.head()

,SECID,day_date,close,value,volume
0,ABIO,2014-06-09,15.000,22579.4,1500.0
1,ABIO,2014-06-10,14.881,25469.9,1700.0
2,ABIO,2014-06-11,14.999,2999.9,200.0
3,ABIO,2014-06-16,15.000,24522.3,1600.0
4,ABIO,2014-06-17,15.120,738392.0,48900.0


In [7]:
df_market = df_tools.merge(df_candles,on='SECID')
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525620 entries, 0 to 525619
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SECID      525620 non-null  object        
 1   BOARDID    525620 non-null  object        
 2   SHORTNAME  525620 non-null  object        
 3   SECNAME    525620 non-null  object        
 4   LOTSIZE    525620 non-null  int64         
 5   SECTYPE    525620 non-null  object        
 6   LISTLEVEL  525620 non-null  int64         
 7   day_date   525620 non-null  datetime64[ns]
 8   close      525620 non-null  float64       
 9   value      525620 non-null  float64       
 10  volume     525620 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 44.1+ MB


### Текстовые данные

In [8]:
df_text['day_date'] = df_text['modified_at'].map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d')) 
df_text = df_text[['day_date','title']].rename(columns={'title':'text'})
df_text = df_text.groupby('day_date',as_index=False)['text'].agg('\n'.join)
df_text.head()

,day_date,text
0,2011-12-17,О конфигурации Фондового рынка Группы ММВБ-РТС...
1,2011-12-29,Об установлении новых условий заключения сдело...
2,2012-01-12,Итоги рынков Группы ММВБ-РТС с 3 по 6 января 2...
3,2012-01-17,Итоги рынков Группы ММВБ-РТС за неделю с 9 по ...
4,2012-01-18,Об установлении новых условий заключения сдело...


## Модель-съедобный вид
Здесь мы превращаем входные данные, в данные которые пойдут непосредственно в регрессоры. А именно, в таблицу со столбцами:
* Целевой инструмент
    * Ид
    * Доп. инфо
    * 30 дней свечей (по отдельному столбцу на день)
* Весь рынок
    * Среднее
    * Дисперсия
    * Квантили
* Текст
    * 30 дней векторов новостей с рынка или с рбк (по отдельному столбцу на день)
* Дата перед предсказываемым днем
    * Год
    * Месяц
    * День месяца
    * День недели

### Автоматизация

In [9]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_text['text'])

TfidfVectorizer()

In [110]:
def transform_ticker(df_market_30_day):
    # ИД и пр. инфо
    df_result = df_market_30_day[[
        'SECID'
        ,'BOARDID'
        ,'SHORTNAME'
        ,'SECNAME'
        ,'LOTSIZE'
        ,'SECTYPE'
        ,'LISTLEVEL'
    ]].drop_duplicates()

    # print(df_market_30_day['day_index'].drop_duplicates().reset_index(drop=True))

    # 30 дней свечей
    df_day_pivot_candles = pd.pivot_table(df_market_30_day
                                          ,values=['close','value','volume']
                                          ,columns='day_index'
                                          ,index=['SECID']
                                          ,fill_value=-1)
    df_day_pivot_candles.columns = ['_'.join(map(lambda x: str(x),col)) for col in df_day_pivot_candles.columns]
    df_day_pivot_candles.reset_index()

    df_result = df_result.merge(df_day_pivot_candles,on='SECID')
    df_result.columns = ['tools_' + col for col in df_result.columns]
    return df_result

def transform_market_statistic(df_market_30_day):
    # Статистика по рынку
    def q25(x): return np.quantile(x,.25)
    def q50(x): return np.quantile(x,.5)
    def q75(x): return np.quantile(x,.75)
    df_market_statistic = pd.pivot_table(
        df_market_30_day
        ,values=['close','value','volume']
        ,columns='day_index'
        ,index=[]
        ,aggfunc=[
            'mean'
            ,'std'
            ,q25
            ,q50
            ,q75
        ]
    )
    df_market_statistic = df_market_statistic.unstack().to_frame().T
    df_market_statistic.columns = ['_'.join(map(lambda x: str(x),col)) for col in df_market_statistic.columns]

    df_market_statistic.columns = ['market_' + col for col in df_market_statistic.columns]

    return df_market_statistic

def transform_text(df_text_30_day):
    vectorize_text = vectorizer.transform(df_text_30_day['text'])
    vectorize_text = vectorize_text.reshape((-1,np.prod(vectorize_text.shape)))
    vectorize_text = vectorize_text.toarray()

    df_vectorize_text = pd.DataFrame(vectorize_text)
    df_vectorize_text.columns = ['text_' + str(col) for col in df_vectorize_text.columns]
    
    return df_vectorize_text

def transfrom_one_period(df_market_30_day,df_text_30_day,td):
    df_result = transform_ticker(df_market_30_day)
    
    df_market_statistic = transform_market_statistic(df_market_30_day)
    df_result['key'] = df_market_statistic['key'] = 0 
    df_result = df_result.merge(df_market_statistic,on='key').drop('key',axis=1)

    # df_vectorize_text = transform_text(df_text_30_day)
    # df_result['key'] = df_vectorize_text['key'] = 0
    # df_result = df_result.merge(df_vectorize_text,on='key').drop('key',axis=1)

    df_result['last_year'] = td.year
    df_result['last_month'] = td.month
    df_result['last_day'] = td.day
    df_result['last_day_of_week'] = td.day_of_week

    return df_result.reset_index(drop=True)


def transform(df_market,df_text):
    DAY_DELTA = 30
    fd = df_market['day_date'].min()
    td = df_market['day_date'].max()
    # df_cld = (pd.date_range(start=fd, end=td, freq='D',name='day_date')
    #           .to_frame()
    #           .reset_index(drop=True))
    df_cld = (df_market['day_date']
              .drop_duplicates()
              .to_frame()
              .reset_index(drop=True))
    if len(df_cld) < DAY_DELTA:
        raise ValueError(f'Слишком мало дат для предсказания. Нужно еще хотя бы {DAY_DELTA - len(df_cld)}.')
    
    df_all = (df_cld
              .merge(df_market,how='left',on='day_date')
              .merge(df_text,how='left',on='day_date')
              .reset_index(drop=True))
    df_all.loc[df_all['text'].isna(),'text'] = ''

    N = len(df_cld)
    dfs = []
    for i in range(N - DAY_DELTA + 1):
        df_cld_period = (df_cld[i:i+DAY_DELTA]
                         .copy()
                         .reset_index(drop=True))
        df_cld_period['day_index'] = df_cld_period.index
        fd_period = df_cld_period['day_date'].min()
        td_period = df_cld_period['day_date'].max()
        
        df_all_in_period = df_cld_period.merge(df_all,on='day_date')
        
        df_market_in_period = df_all_in_period[[
            'day_index'
            ,'SECID'
            ,'BOARDID'
            ,'SHORTNAME'
            ,'SECNAME'
            ,'LOTSIZE'
            ,'SECTYPE'
            ,'LISTLEVEL'
            ,'close'
            ,'value'
            ,'volume'
        ]].copy()
        
        df_text_in_period = df_all_in_period[[
            'day_index'	
            ,'text'
        ]].copy()

        df_transform_period = transfrom_one_period(df_market_in_period,df_text_in_period,td_period)
        dfs.append(df_transform_period)
    
    # return pd.concat(dfs,ignore_index=True)
    return dfs
    
df_cld_test = (df_market['day_date']
               .drop_duplicates()
               .reset_index(drop=True)
               .to_frame()
               .iloc[:31])
df_market_test = df_cld_test.merge(df_market,on='day_date')
df_text_test = df_cld_test.merge(df_text,on='day_date') 
res = transform(df_market_test,df_text_test)
res

[    tools_SECID tools_BOARDID tools_SHORTNAME                  tools_SECNAME  \
 0          ABIO          TQBR      iАРТГЕН ао                   ПАО "Артген"   
 1          ABRD          TQBR      АбрауДюрсо             Абрау-Дюрсо ПАО ао   
 2          AFKS          TQBR      Система ао           АФК "Система" ПАО ао   
 3          AFLT          TQBR        Аэрофлот   Аэрофлот-росс.авиалин(ПАО)ао   
 4          AKRN          TQBR           Акрон                   Акрон ПАО ао   
 ..          ...           ...             ...                            ...   
 182        RTGZ          TQBR       ГР Ростов  Газпром газорасп Р-н-Д ПАО ао   
 183        KUZB          TQBR      КузнецкийБ        Банк "Кузнецкий" ПАО ао   
 184       VRSBP          TQBR      ТНСэнВор-п      ТНС энерго Воронеж ПАО ап   
 185       VSYDP          TQBR      ВыбСудЗ ап    Выборгский Суд.Завод ПАО ап   
 186        ZVEZ          TQBR       ЗВЕЗДА ао                  ЗВЕЗДА ПАО ао   
 
      tools_LOTSIZE tools_

In [111]:
for elem in zip(res[0].columns.to_numpy(),res[1].columns.to_numpy()):
    if elem[0] != elem[1]:
        print(elem)
    # print(elem)

In [87]:
pd.concat(res,ignore_index=True)

,tools_SECID,tools_BOARDID,tools_SHORTNAME,tools_SECNAME,tools_LOTSIZE,tools_SECTYPE,tools_LISTLEVEL,tools_close_2,tools_close_3,tools_close_6,...,market_std_1_volume,tools_close_0,tools_value_0,tools_volume_0,market_mean_0_close,market_mean_0_value,market_mean_0_volume,market_std_0_close,market_std_0_value,market_std_0_volume
0,ABIO,TQBR,iАРТГЕН ао,"ПАО ""Артген""",10.0,1,2.0,14.142,14.06300,14.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRD,TQBR,АбрауДюрсо,Абрау-Дюрсо ПАО ао,10.0,1,3.0,144.980,141.54000,139.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFKS,TQBR,Система ао,"АФК ""Система"" ПАО ао",100.0,1,1.0,47.790,48.00000,46.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFLT,TQBR,Аэрофлот,Аэрофлот-росс.авиалин(ПАО)ао,10.0,1,1.0,59.340,58.26000,57.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AKRN,TQBR,Акрон,Акрон ПАО ао,1.0,1,2.0,1254.900,1247.30000,1215.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,KCHEP,TQBR,КамчатЭ ап,Камчатскэнерго ПАО ап,10000.0,2,3.0,-1.000,-1.00000,-1.00,...,4.042333e+09,0.06376,3825.6,60000.0,512.873718,3.030425e+08,5.213061e+08,1522.995964,1.210085e+09,4.572753e+09
548,RTGZ,TQBR,ГР Ростов,Газпром газорасп Р-н-Д ПАО ао,1.0,1,3.0,-1.000,-1.00000,11200.00,...,4.042333e+09,-1.00000,-1.0,-1.0,512.873718,3.030425e+08,5.213061e+08,1522.995964,1.210085e+09,4.572753e+09
549,KAZTP,TQBR,Куйбазот-п,Куйбышевазот ПАО ап,10.0,2,3.0,-1.000,-1.00000,54.59,...,4.042333e+09,-1.00000,-1.0,-1.0,512.873718,3.030425e+08,5.213061e+08,1522.995964,1.210085e+09,4.572753e+09
550,KUZB,TQBR,КузнецкийБ,"Банк ""Кузнецкий"" ПАО ао",10000.0,1,3.0,-1.000,0.01603,-1.00,...,4.042333e+09,-1.00000,-1.0,-1.0,512.873718,3.030425e+08,5.213061e+08,1522.995964,1.210085e+09,4.572753e+09


# Создание модели